# Single Step Prediction

Import relevant modules

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pywt
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
keras.backend.clear_session()

Read coefficient, in this case a3 from DWT

In [ ]:
data_all = pd.read_csv(r'C:\Users\AliMS\Documents\project\code\data\10_2018_dwt_a3.csv', usecols = ['a3'])

Normalise data

In [ ]:
scaler = MinMaxScaler()
data_all = scaler.fit_transform(data_all)

Select data proportions. Select how far ahead to predict using 'steps_ahead'

In [ ]:
train_size = int(data_all.shape[0]*0.9)
data_train = data_all[:train_size, :]
data_test = data_all[train_size:, :]
steps_ahead = 1

Assign train, validate and test data

In [ ]:
X_train, Y_train = data_train[: - steps_ahead, :], data_train[steps_ahead:, :]
X_test, Y_test = data_test[: - steps_ahead, :], data_test[steps_ahead:, :]
X_all, Y_all = data_all[: - steps_ahead, :], data_all[steps_ahead:, :]

Reshape arrays

In [ ]:
X_train = X_train.reshape(-1, 1, 1)
X_test = X_test.reshape(-1, 1, 1)
Y_train = Y_train.reshape(-1, 1, 1)
X_all = X_all.reshape(-1, 1, 1)

Create model. In this example LSTM, replace with SimpleRNN or GRU

In [ ]:
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences = True, input_shape = [None, 1]),
    keras.layers.LSTM(320, return_sequences = True),
    keras.layers.Dense(1, activation='sigmoid'),
])

Select optimizer and compile

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss = 'mse', optimizer = optimizer)

Train model

In [ ]:
model.fit(X_train, Y_train, epochs = 20, validation_split = 0.1, verbose=1)

Score model

In [ ]:
score = model.evaluate(X_test, Y_test)
print('Score: {}'.format(score))

Predict, reshape and denormalise data

In [ ]:
Y_pred = model.predict(X_all)
Y_pred = Y_pred.reshape(-1,1)
Y_pred = scaler.inverse_transform(Y_pred)
Y_all = scaler.inverse_transform(Y_all)

In [ ]:
data_all = pd.DataFrame(data_all)

Correct length

In [ ]:
Y_pred = pd.DataFrame(Y_pred)
for steps in range (1, steps_ahead + 1):
    Y_pred = Y_pred.rename(columns = {0:'a3'})
    Y_pred = Y_pred.append(Y_pred.iloc[0], ignore_index=True)
    Y_pred.at[0,'a3'] = Y_pred.at[1,'a3']
Y_pred.shape

Write to csv file

In [ ]:
Y_pred.to_csv(r'C:\Users\AliMS\Documents\project\code\data\10_2018_pred_dwt_a3.csv')